# Azure AI Document Intelligence を使用した Semantic Chunker

[Azure AI Document Intelligence](https://azure.microsoft.com/products/ai-services/ai-document-intelligence) は LangChain とネイティブに統合され、データの取り込み処理において精度を向上させることができます。

まず、`AzureAIDocumentIntelligenceLoader` を使用して文書を **Markdown** 形式で読み取ります。その後、`MarkdownHeaderTextSplitter` が意味の理解に基づいてテキストをチャンクに分割します。こうすることで各チャンク内で意味の一貫性が維持されるという明確な利点があります。

https://learn.microsoft.com/azure/ai-services/document-intelligence/concept-retrieval-augumented-generation?view=doc-intel-4.0.0

In [ ]:
!pip install azure-ai-documentintelligence langchain langchain-community tiktoken

In [ ]:
import azure.ai.documentintelligence
print("Azure Document Intelligence version: ", azure.ai.documentintelligence.__version__)
import langchain
print("Langchain version: ", langchain.__version__)

In [ ]:
# Using SDK targeting 2023-10-31-preview, make sure your resource is in one of these regions: East US, West US2, West Europe
# pip install azure-ai-documentintelligence==1.0.0b1
# pip install langchain langchain-community azure-ai-documentintelligence

from azure.ai.documentintelligence import DocumentIntelligenceClient

endpoint = "<your-endpoint>"
key = "<your-key>"

from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
 
# Azure AI Document Intelligence を起動して、ドキュメントを読み込みます。ドキュメントを読み込むには、file_path または url_path を指定します。
loader = AzureAIDocumentIntelligenceLoader(file_path="./pdf/源実朝 - Wikipedia.pdf", api_key = key, api_endpoint = endpoint, api_model="prebuilt-layout", mode="markdown")
docs = loader.load()

docs[0].page_content

## Markdown ファイルとして書き出し
ページコンテンツをそのまま Markdown ファイルとして書き出します。VSCode があれば、`.md` ファイルのプレビューがリアルタイムでできます。

In [ ]:
with open('page_content.md', 'w') as f:
    f.write(docs[0].page_content)

## MarkdownHeaderTextSplitter
チャンク化戦略において、文書自体の構造を特に重視する場合、文書をマークダウン化し、特定のヘッダーグループごとにチャンクを作成することは直感的です。`MarkdownHeaderTextSplitter` は指定されたヘッダーのセットによってマークダウン ファイルが分割されます。

In [ ]:
# マークダウンのヘッダーに基づき、ドキュメントをチャンクに分割する。
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
]
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers = False)
 
docs_string = docs[0].page_content
splits = text_splitter.split_text(docs_string)
splits

In [ ]:
len(splits)

## メタデータのみを抽出

In [ ]:
for i, text in enumerate(splits):
    print(i, text.metadata)

## ドキュメントのトークン数を計測
OpenAI モデルのトークン数を正確に数えるためには、[tiktoken](https://github.com/openai/tiktoken) ライブラリを利用します。モデルごとに利用するトークナイザーが異なるため詳しくは[こちら](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb)を参照してください。

In [ ]:
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")
#チャンクごとのトークン数確認
tokencounter = 0
for i, text in enumerate(splits):
    tokenlength = len(enc.encode(text.page_content))
    print(i, tokenlength)
    tokencounter = tokencounter + tokenlength

print(tokencounter)

### 分割の確認


In [ ]:
splits[0]

### 表形式の確認

In [ ]:
splits[9]

##  RecursiveCharacterTextSplitter
各マークダウン グループ内で、必要なテキスト スプリッターを適用できます。
`RecursiveCharacterTextSplitter` は、一般的なテキストに推奨されます。文字のリストによってパラメータ化されます。チャンクが十分に小さくなるまで、順番に分割しようとします。デフォルトのリストは ["\n\n", "\n", " ", ""] です。これには、すべての段落 (次に文、そして単語) をできるだけ長くまとめて保持しようとする効果があります。これらの段落は、一般的に意味的に関連性が最も強いテキストであると思われるためです。


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size = 1000
chunk_overlap = 100
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits_2nd = text_splitter.split_documents(splits)
splits_2nd

In [ ]:
#チャンクごとのトークン数確認
tokencounter = 0
for i, text in enumerate(splits_2nd):
    tokenlength = len(enc.encode(text.page_content))
    print(i, tokenlength)
    tokencounter = tokencounter + tokenlength

print(tokencounter)

分割の確認

In [ ]:
splits_2nd[3]

In [ ]:
splits_2nd[4]